# 4.7 建立堆疊的函式

In [5]:
# --- 第 1 部分 ---
# 匯入函式庫
import numpy as np
from sklearn.model_selection import KFold
from copy import deepcopy

In [6]:
# --- 第 2 部分 ---
class StackingRegressor():
    def __init__(self, learners):
        # 接收基學習器、超學習器、以及堆疊中每一層分別有多少學習器
        # 複製學習器
        self.level_sizes = []
        self.learners = []
        for learning_level in learners:
            self.level_sizes.append(len(learning_level))
            level_learners = []
            for learner in learning_level:
                level_learners.append(deepcopy(learner))
            self.learners.append(level_learners)
# --- 第 3 部分 ---
    # fit 函式
    # 用第i-1層的基學習器預測值來訓練第i層的基學習器
    def fit(self, x, y):
        # 第1層基學習器的訓練資料即為原始資料
        meta_data = [x]
        meta_targets = [y]
        for i in range(len(self.learners)):
            level_size = self.level_sizes[i]

            # 建立第i層預測值的儲存空間
            data_z = np.zeros((level_size, len(x)))
            target_z = np.zeros(len(x))

            # 取得第i層訓練資料集
            train_x = meta_data[i]
            train_y = meta_targets[i]

            # 建立交叉驗證
            KF = KFold(n_splits=5)
            meta_index = 0
            for train_indices, test_indices in KF.split(x):
                for j in range(len(self.learners[i])):
                    # 使用前K-1折訓練第j個基學習器
                    learner = self.learners[i][j]
                    learner.fit(train_x[train_indices],train_y[train_indices])
                    # 使用第K折驗證第j個基學習器
                    predictions = learner.predict(train_x[test_indices])
                    # 儲存第K折第j個基學習器預測結果
                    data_z[j][meta_index:meta_index+len(test_indices)] = predictions

                target_z[meta_index:meta_index + len(test_indices)] = train_y[test_indices]
                meta_index += len(test_indices)

            # 儲存第i層基學習器的預測結果
            # 作為第i+1層基學習器的訓練資料
            data_z = data_z.transpose()
            meta_data.append(data_z)
            meta_targets.append(target_z)


            # 使用完整的訓練資料來訓練基學習器
            for learner in self.learners[i]:
                learner.fit(train_x, train_y)
# --- 第 4 部分 ---
    # predict 函式
    def predict(self, x):

        # 儲存每一層的預測
        meta_data = [x]
        for i in range(len(self.learners)):
            level_size = self.level_sizes[i]

            data_z = np.zeros((level_size, len(x)))

            test_x = meta_data[i]

            for j in range(len(self.learners[i])):

                learner = self.learners[i][j]
                predictions = learner.predict(test_x)
                data_z[j] = predictions

            # 儲存第i層基學習器的預測結果
            # 作為第i+1層基學習器的輸入
            data_z = data_z.transpose()
            meta_data.append(data_z)

        # 傳回預測結果
        return meta_data

In [3]:
# meta_data[0][0]==test_x[0]

NameError: name 'meta_data' is not defined

In [7]:
# 匯入函式庫與資料集
from sklearn.datasets import load_diabetes
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn import metrics
diabetes = load_diabetes()

train_x, train_y = diabetes.data[:400], diabetes.target[:400]
test_x, test_y = diabetes.data[400:], diabetes.target[400:]

base_learners = []

knn = KNeighborsRegressor(n_neighbors=5)
base_learners.append(knn)

dtr = DecisionTreeRegressor(max_depth=4, random_state=123456)
base_learners.append(dtr)

ridge = Ridge()
base_learners.append(ridge)

meta_learner = LinearRegression()

# 傳入我們建立的函式
sc = StackingRegressor([[knn,dtr,ridge],[meta_learner]])

# 訓練、預測
sc.fit(train_x, train_y)
meta_data = sc.predict(test_x)

# 衡量基學習器跟集成後效能
base_errors = []
base_r2 = []
for i in range(len(base_learners)):
    learner = base_learners[i]
    predictions = meta_data[1][:,i]
    err = metrics.mean_squared_error(test_y, predictions)
    r2 = metrics.r2_score(test_y, predictions)
    base_errors.append(err)
    base_r2.append(r2)

err = metrics.mean_squared_error(test_y, meta_data[-1])
r2 = metrics.r2_score(test_y, meta_data[-1])

# 顯示結果
print('ERROR  R2  Name')
print('-'*20)
for i in range(len(base_learners)):
    e = base_errors[i]
    r = base_r2[i]
    b = base_learners[i]
    print(f'{e:.1f} {r:.2f} {b.__class__.__name__}')
print(f'{err:.1f} {r2:.2f} Ensemble')


ERROR  R2  Name
--------------------
2697.8 0.51 KNeighborsRegressor
3142.5 0.43 DecisionTreeRegressor
2564.8 0.54 Ridge
2066.6 0.63 Ensemble
